# Paper figures

In this notebook all the figures from the paper are generated.

Download the data if you haven't done so, see [README.md](README.md) for the download instructions.

In [ ]:
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
from funcs import constants
import funcs
import os

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

os.makedirs("figures", exist_ok=True)
golden_mean = (np.sqrt(5) - 1) / 2  # Aesthetic ratio
fig_width_pt = 246.0  # Columnwidth
inches_per_pt = 1.0 / 72.27  # Convert pt to inches
fig_width = fig_width_pt * inches_per_pt
fig_height = fig_width * golden_mean  # height in inches
fig_size = [fig_width, fig_height]
fig_width2, fig_height2 = 2.7, 3

params = {
    "backend": "ps",
    "axes.labelsize": 10,
    "font.size": 10,
    "legend.fontsize": 8,
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
    "text.usetex": True,
    "figure.figsize": fig_size,
    "font.family": "serif",
    "font.serif": "Computer Modern Roman",
    "legend.frameon": True,
    "savefig.dpi": 150,
    "figure.dpi": 150,
}

plt.rcParams.update(params)
plt.rc(
    "text.latex",
    preamble=[r"\usepackage{color}", r"\usepackage{bm}", r"\usepackage{siunitx}"],
)

import matplotlib.colors as colors


class HistogramNormalize(colors.Normalize):
    def __init__(self, data, vmin=None, vmax=None, mixing_degree=1):
        self.mixing_degree = mixing_degree
        if vmin is not None:
            data = data[data >= vmin]
        if vmax is not None:
            data = data[data <= vmax]

        self.sorted_data = np.sort(data.flatten())
        colors.Normalize.__init__(self, vmin, vmax)

    def __call__(self, value, clip=None):
        hist_norm = np.ma.masked_array(
            np.searchsorted(self.sorted_data, value) / len(self.sorted_data)
        )
        linear_norm = super().__call__(value, clip)
        return self.mixing_degree * hist_norm + (1 - self.mixing_degree) * linear_norm


def open_mtx(fname):
    with open(fname, "rb") as f:
        line1 = f.readline()
        *shape, entry_size = [int(i) for i in f.readline().split()]
        print("shape: ", shape)
        data = np.fromfile(f, (np.float64 if entry_size == 8 else np.float32))
        data = data.reshape(shape).squeeze()

    keys = [
        "units",
        "plot_settings",
        "xname",
        "xmin",
        "xmax",
        "yname",
        "ymin",
        "ymax",
        "zname",
        "zmin",
        "zmax",
    ]
    values = line1.decode("utf-8").replace("\n", "").split(",")
    meta_data = dict(zip(keys, values))
    meta_data = {}
    for key, val in zip(keys, values):
        try:
            meta_data[key] = float(val)
        except:
            meta_data[key] = val
    return meta_data, data


def add_energy_gs(df):
    flux_quantum_over_2pi = constants.hbar / (2 * constants.eV) / (constants.eV * 1e6)
    df["E"] = df["currents"].apply(np.cumsum)
    df["E"] *= flux_quantum_over_2pi

    df["phase_gs_argsort"] = df["E"].apply(np.argsort)
    df["phase_gs_sort"] = df["E"].apply(np.sort)

    df["phase_gs_arg"] = df["E"].apply(np.argmin)
    df["phase_gs"] = [row["phases"][row["phase_gs_arg"]] for i, row in df.iterrows()]

    # Move the phase_gs from -π to +π if they are within the tolerance
    tol = np.diff(df["phases"].iloc[0]).max()
    df["phase_gs"] = [
        -row["phase_gs"] if row["phase_gs"] < -(np.pi - tol) else row["phase_gs"]
        for i, row in df.iterrows()
    ]
    return df

# Figure 1

In [ ]:
from matplotlib._png import read_png
from matplotlib.offsetbox import TextArea, DrawingArea, OffsetImage, AnnotationBbox

meta, dat = open_mtx("data/experimental_data/fig1.mtx")
meta["ymax"], meta["ymin"] = meta["ymin"], meta["ymax"]
fig = plt.figure(figsize=(fig_width, fig_height))
ax = fig.add_axes([0.1, 0.2, 0.71, 0.85])
extent = (meta["xmin"], meta["xmax"], meta["ymin"], meta["ymax"])
norm = HistogramNormalize(dat, vmin=0, vmax=16, mixing_degree=0.5)
kwargs = dict(
    extent=extent,
    cmap="viridis",
    aspect="auto",
    norm=norm,
    interpolation="nearest",
    origin="lower",
    vmax=norm.vmax,
)
im = ax.imshow(np.rot90(dat), **kwargs)
ax.set_xlabel("$B$ [$T$]")
ax.set_ylabel(r"$I_\textrm{bias}$ [nA]")
cax = fig.add_axes([0.83, 0.2, 0.04, 0.85])
cb = fig.colorbar(
    im, cax=cax, extend="max", label=r"$\mathrm{d}V/\mathrm{d}I$ [k$\Omega$]"
)
cbar_ticks = [0, 5, 10, 15]
cb.set_ticks(cbar_ticks)

fig.savefig(
    "figures/fig1_2.pdf", format="pdf", dpi=150, bbox_inches="tight", pad_inches=0
)
plt.show()

# Figure 2

In [ ]:
fig, axs = plt.subplots(
    2, 2, sharey=True, sharex=True, figsize=(fig_width, fig_height * 1.7)
)
plt.subplots_adjust(
    bottom=0.2, left=0.125, right=0.80, top=0.8, hspace=0.25, wspace=0.10
)


exp = [
    open_mtx("data/experimental_data/fig2a.mtx"),
    open_mtx("data/experimental_data/fig2b.mtx"),
    open_mtx("data/experimental_data/fig2c.mtx"),
    open_mtx("data/experimental_data/fig2d.mtx"),
]

for i, ax in enumerate(axs.reshape(-1)):
    meta, dat = exp[i]
    extent = (meta["xmin"], meta["xmax"], meta["ymin"], meta["ymax"])
    print(extent)
    data = dat.T / 1000
    if i == 0:
        norm = HistogramNormalize(data, vmin=0, vmax=22, mixing_degree=0.6)
        kwargs = dict(
            extent=extent,
            cmap="viridis",
            aspect="auto",
            norm=norm,
            interpolation="nearest",
            origin="lower",
        )
        im = ax.imshow(data, **kwargs)
    else:
        ax.imshow(data, **kwargs)
    ax.set_xticks([0, 0.2, 0.4])
    ax.set_yticks([0, 1, 2, 3])
    ax.set_ylim(0, 3.6)
    ax.set_xlim(0, 0.45)
    ax.set_xlabel("")
    if i in [0, 2]:
        ax.set_ylabel(r"$I_\textrm{bias}$ [nA]")
    if i in [2, 3]:
        ax.set_xlabel("$B$ [$T$]")

    names = [
        r"(a) $2.64\;\mathrm{V}$",
        r"(b) $2.66\;\mathrm{V}$",
        r"(c) $2.68\;\mathrm{V}$",
        r"(d) $2.70\;\mathrm{V}$",
    ]

    ax.text(
        0.5,
        1.1,
        names[i],
        horizontalalignment="center",
        verticalalignment="center",
        transform=ax.transAxes,
        fontdict={"color": "k", "size": 10},
    )

cax = fig.add_axes([0.83, 0.2, 0.04, 0.6])
cbar_ticks = [0, 5, 10, 15, 20, 25]
cb = fig.colorbar(
    im, cax=cax, extend="max", label=r"$\mathrm{d}V/\mathrm{d}I$ [k$\Omega$]"
)
cb.set_ticks(cbar_ticks)
fig.savefig("figures/fig2.pdf", format="pdf", dpi=150)
plt.show()

# Figure 3

In [ ]:
fig, axs = plt.subplots(
    5, 1, sharey=False, sharex=True, figsize=(fig_width, fig_height * 1.6)
)
plt.subplots_adjust(
    bottom=0.2, left=0.125, right=0.80, top=0.9, hspace=0.150, wspace=0.10
)


exp = [
    open_mtx("data/experimental_data/fig3a.mtx"),
    open_mtx("data/experimental_data/fig3b.mtx"),
    open_mtx("data/experimental_data/fig3c.mtx"),
    open_mtx("data/experimental_data/fig3d.mtx"),
    open_mtx("data/experimental_data/fig3e.mtx"),
]


fig.text(0.00, 0.6, r"$I_\textrm{bias}$ [nA]", va="center", rotation="vertical")
for i, ax in enumerate(axs.reshape(-1)):
    meta, dat = exp[i]
    meta["ymax"], meta["ymin"] = meta["ymin"], meta["ymax"]
    if i == 0:
        ymax_new = 3.4
        ax.set_yticks([0, 1.5, 3])
    if i == 1:
        ymax_new = 2
        ax.set_yticks([0.0, 0.9, 1.8])
    if i == 2:
        ymax_new = 0.8
        ax.set_yticks([0, 0.3, 0.6])
    if i == 3:
        ymax_new = 0.5
        ax.set_yticks([0, 0.2, 0.4])
    if i == 4:
        ymax_new = 0.5
        ax.set_yticks([0, 0.2, 0.4])

    extent = (
        float(meta["xmin"]),
        float(meta["xmax"]),
        float(meta["ymin"]),
        float(meta["ymax"]),
    )

    data = np.rot90(dat) / 1000

    if i == 0:
        norm = HistogramNormalize(data, vmax=55, vmin=0, mixing_degree=0.5)
        kwargs = dict(
            extent=extent,
            cmap="viridis",
            aspect="auto",
            norm=norm,
            interpolation="nearest",
            origin="lower",
        )
        im = ax.imshow(data, **kwargs)
    else:
        ax.imshow(data, **kwargs)

    ax.set_xticks([0.6, 0.7, 0.8])

    ax.set_ylim(meta["ymin"], ymax_new)
    ax.set_xlim(meta["xmin"], 0.88)
    ax.set_xlabel("")

    if i == 4:
        ax.set_xlabel(r"$V_\textrm{gate}$ [$V$]")

    names = [
        r"(a)  $0\;\mathrm{mT}$",
        r"(b) $100\;\mathrm{mT}$",
        r"(c) $200\;\mathrm{mT}$",
        r"(d) $300\;\mathrm{mT}$",
        r"(e) $400\;\mathrm{mT}$",
    ]

    ax.text(
        0.09,
        0.72,
        names[i],
        horizontalalignment="left",
        verticalalignment="center",
        transform=ax.transAxes,
        fontdict={"color": "#330066", "size": 11},
    )

cax = fig.add_axes([0.83, 0.2, 0.04, 0.7])
cbar_ticks = [0, 10, 20, 30, 40, 50]
cb = fig.colorbar(
    im, cax=cax, extend="max", label=r"$\mathrm{d}V/\mathrm{d}I$ [k$\Omega$]"
)
cb.set_ticks(cbar_ticks)
fig.savefig("figures/fig3.pdf", format="pdf", dpi=150, bbox_inches="tight")
plt.show()

# Figure 4

In [ ]:
meV = 20
salt = 7
from scipy.signal import argrelextrema

df = pd.read_hdf(
    "data/I_c(B_x)_mu10,20meV_disorder0,75meV_T0.1K_all_combinations_of_effects.hdf"
)

df = add_energy_gs(df)

gb = df.groupby(["orbital", "g", "alpha", "mu", "disorder", "salt"])
total = gb.get_group((True, 50, 20, meV, 0, 0))
B_x = total["B_x"]
current = lambda orbital=True, g=50, alpha=20, mu=10, disorder=0, salt=0: gb.get_group(
    (orbital, g, alpha, mu, disorder, salt)
)["current_c"].values

phase = lambda orbital=True, g=50, alpha=20, mu=10, disorder=0, salt=0: gb.get_group(
    (orbital, g, alpha, mu, disorder, salt)
)["phase_gs"].values


def plot_curves(ax, mu, plot=current, salt=0):
    args = {"lw": 1}
    ax.plot(B_x, plot(mu=mu), label=r"orbital + SOI", c="k", ls="-", **args)
    ax.plot(B_x, plot(orbital=False, mu=mu), label=r"SOI", c="g", ls="--", **args)
    ax.plot(B_x, plot(alpha=0, mu=mu), label=r"orbital", c="m", ls="-.", **args)
    ax.plot(
        B_x,
        plot(orbital=False, alpha=0, mu=mu),
        label=r"Zeeman only",
        c="c",
        ls=":",
        **args
    )
    if mu == 10:
        ax.plot(
            np.nan,
            np.nan,
            label=r"orbital + SOI + disorder, $l_\textrm{mfp}=250$ nm (6x)",
            c="r",
            lw=1.8,
        )  # just for the legend
    else:
        y = plot(mu=mu, disorder=75, salt=salt)
        if plot == current:
            y *= 6
        ax.plot(B_x, y, label=r"disorder 2", c="r", lw=1.8)
    ax.set_xlim(B_x.min(), B_x.max())

    return ax


fig, axs = plt.subplots(
    2,
    2,
    sharey=False,
    sharex=True,
    figsize=(2 * fig_width, 1.7 * fig_height),
    gridspec_kw={"height_ratios": [2, 1]},
)

plt.subplots_adjust(
    bottom=0.2, left=0.125, right=0.90, top=0.9, hspace=0.15, wspace=0.25
)


ymax = 22
axs[0][0] = plot_curves(axs[0][0], 10, salt=salt)
axs[0][0].set_yticks(np.arange(0, ymax, 5))
axs[0][0].set_ylim(0, ymax)
axs[0][0].set_xlabel("")
axs[0][0].set_ylabel("$I_c$ [nA]")

ymax = 120
axs[0][1] = plot_curves(axs[0][1], meV, salt=salt)
axs[0][1].set_yticks(np.arange(0, ymax, 20))
axs[0][1].set_ylim(0, ymax)
axs[0][1].set_ylabel("$I_c$ [nA]")


for i in np.argwhere(
    np.abs(np.diff(gb.get_group((False, 50, 0, 10, 0, 0))["phase_gs"].values)) > 3
):
    x = B_x.iloc[i[0]]
    y1, y2 = 0, ymax
    axs[0][0].plot((x, x), (y1, y2), "c--")

for i in np.argwhere(
    np.abs(np.diff(gb.get_group((False, 50, 0, meV, 0, 0))["phase_gs"].values)) > 3
):
    x = B_x.iloc[i[0]]
    y1, y2 = 0, ymax
    axs[0][1].plot((x, x), (y1, y2), "c--")

axs[1][0] = plot_curves(axs[1][0], 10, phase, salt=salt)
axs[1][0].set_yticks([-3.14, 0, 3.14])
axs[1][0].set_yticklabels([r"$-\pi$", 0, r"$\pi$"])
axs[1][0].set_xlabel("$B$ [$T$]")
axs[1][0].set_ylabel(r"$\theta_\textrm{gs}$")

axs[1][1] = plot_curves(axs[1][1], meV, phase, salt=salt)
axs[1][1].set_yticks([-3.14, 0, 3.14])
axs[1][1].set_yticklabels([r"$-\pi$", 0, r"$\pi$"])
axs[1][1].set_xlabel("$B$ [$T$]")
axs[1][1].set_ylabel(r"$\theta_\textrm{gs}$")

for i, ax in enumerate(axs.reshape(-1)):
    ax.text(
        -0.20,
        0.98,
        "({})".format("acbd"[i]),
        horizontalalignment="left",
        verticalalignment="center",
        transform=ax.transAxes,
        fontdict={"color": "#330066", "size": 11},
    )

leg0 = axs[0][0].legend(
    loc="upper center", bbox_to_anchor=(1.15, 1.34), ncol=3, prop={"size": 8}
)

axs[0][0].set_xlim(0, 0.5)

fig.savefig(
    "figures/fig4.pdf".format(meV, salt),
    format="pdf",
    dpi=150,
    bbox_inches="tight",
    additional_artists=[leg0],
)
plt.show()

# Figure 5

In [ ]:
import matplotlib.patheffects as PathEffects

fig, axs = plt.subplots(
    2, 1, sharey=False, sharex=True, figsize=(fig_width, fig_height * 1.8)
)
plt.subplots_adjust(bottom=0.2, left=0.1, right=0.80, top=0.9, hspace=0.10, wspace=0.50)
df = pd.read_hdf("data/I_c(B_x,_V)_gate160nm_mu20meV_disorder0,75meV_T0.1K.hdf")
df["mu_gate"] = df["mu"] - df["V"]

gb = df.groupby(["disorder", "gate_size"])
gr = gb.get_group((75, 160))

data = gr.pivot(index="B_x", columns="mu_gate", values="current_c").T

exp_data = np.loadtxt("data/experimental_data/fig5.txt") * 1e9
extent0 = (0, 0.420, 0.5750, 1.0750)
norm0 = HistogramNormalize(np.array(exp_data), mixing_degree=0.5)
kwargs = dict(cmap="viridis", aspect="auto", origin="lower", interpolation="nearest")

im0 = axs[0].imshow(exp_data.T, extent=extent0, norm=norm0, **kwargs)
axs[0].set_ylabel(r"$V_\textrm{gate}$ [V]")
axs[0].set_ylim((0.6, 1))
axs[0].set_xlim((0, 0.42))
axs[0].set_yticks([0.6, 0.7, 0.8, 0.9, 1])
cax0 = fig.add_axes([0.83, 0.565, 0.04, 0.335])
cbar_ticks = [0, 1, 2, 3, 4, 5]
cb0 = fig.colorbar(im0, cax=cax0, label=r"$I_{\textrm{sw}}$ [nA]")
cb0.set_ticks(cbar_ticks)

extent1 = (gr.min()["B_x"], gr.max()["B_x"], gr.min()["mu_gate"], gr.max()["mu_gate"])
norm1 = HistogramNormalize(np.array(data), mixing_degree=0.5)
kwargs["interpolation"] = "spline16"
im1 = axs[1].imshow(data, extent=extent1, norm=norm1, **kwargs)
axs[1].set_xlabel("$B$ [$T$]")
axs[1].set_ylabel(r"$\mu_{\textrm{gate}}$ [meV]")
axs[1].set_xticks([0, 0.1, 0.2, 0.3, 0.4])

# axs[1].set_ylim((5, 15))
# yticks = np.arange(5, 15, 3)
# axs[1].set_yticks(yticks)
cax1 = fig.add_axes([0.83, 0.20, 0.04, 0.335])
cb1 = fig.colorbar(im1, cax=cax1, label=r"$I_c$ [nA]")
# cb1.set_ticks(cbar_ticks)
names = [
    "experiment",
    "theory",
]
for i, ax in enumerate(axs):
    ax.text(
        -0.22,
        0.98,
        "({})".format("ab"[i]),
        horizontalalignment="right",
        verticalalignment="center",
        transform=ax.transAxes,
        fontdict={"size": 13, "color": "k"},
        fontweight="bold",
    )

    ax.text(
        0.98,
        0.93,
        names[i],
        horizontalalignment="right",
        verticalalignment="center",
        transform=ax.transAxes,
        fontdict={"color": "w", "size": 11},
    )

fig.savefig("figures/fig5.pdf", format="pdf", dpi=150, bbox_inches="tight")

In [ ]:
df.disorder.unique()

# Supplementary figure 1c

In [ ]:
meta, dat = open_mtx("data/experimental_data/sup_fig1c.mtx")
dat = np.rot90(dat) / 1000
meta["ymax"], meta["ymin"] = meta["ymin"], meta["ymax"]
fig = plt.figure(figsize=(fig_width, fig_height))
ax = fig.add_axes([0.1, 0.2, 0.71, 0.85])
extent = (meta["xmin"], meta["xmax"], meta["ymin"], meta["ymax"])
norm = HistogramNormalize(dat, vmin=0, vmax=200, mixing_degree=0.5)
kwargs = dict(
    extent=extent,
    cmap="viridis",
    aspect="auto",
    norm=norm,
    interpolation="nearest",
    origin="lower",
    vmax=norm.vmax,
)
im = ax.imshow(dat, **kwargs)
ax.set_xlabel(r"$V_\textrm{gate}$ [$T$]")
ax.set_ylabel(r"$I_\textrm{bias}$ [nA]")
ax.set_xticks([0, 0.5, 1, 1.5])
ax.set_yticks([])
cax = fig.add_axes([0.83, 0.2, 0.04, 0.85])
cb = fig.colorbar(
    im, cax=cax, extend="max", label=r"$\mathrm{d}V/\mathrm{d}I$ [k$\Omega$]"
)
cb.set_ticks(np.arange(0, 240, 40))

fig.savefig(
    "figures/sup_fig1c.pdf", format="pdf", dpi=150, bbox_inches="tight", pad_inches=0
)
plt.show()

# Supplementary figure 2a

In [ ]:
meta, dat = open_mtx("data/experimental_data/sup_fig2a.mtx")
dat = np.rot90(dat)
meta["ymax"], meta["ymin"] = meta["ymin"], meta["ymax"]
fig = plt.figure(figsize=(fig_width2, fig_height2))
ax = fig.add_axes([0.1, 0.2, 0.71, 0.85])
extent = (meta["xmin"], meta["xmax"], meta["ymin"], meta["ymax"])
norm = HistogramNormalize(dat, vmin=-10, vmax=60, mixing_degree=0.5)
kwargs = dict(
    extent=extent,
    cmap="viridis",
    aspect="auto",
    norm=norm,
    interpolation="nearest",
    origin="lower",
    vmax=norm.vmax,
)
im = ax.imshow(dat, **kwargs)
ax.set_xlabel(r"$B$ [$T$]")
ax.set_ylabel(r"$I_\textrm{bias}$ [nA]")
ax.set_xticks([0, 0.1, 0.2, 0.3, 0.4, 0.5])
ax.set_yticks([0, 0.5, 1, 1.5])
cax = fig.add_axes([0.83, 0.2, 0.04, 0.85])
cb = fig.colorbar(
    im, cax=cax, extend="max", label=r"$\mathrm{d}V/\mathrm{d}I$ [k$\Omega$]"
)
cb.set_ticks(np.arange(0, 140, 20))

# fig.savefig('figures/sup_fig2a.pdf', format='pdf', dpi=150,
#            bbox_inches='tight', pad_inches=0)
plt.show()

# Supplementary figure 2c

In [ ]:
meta, dat = open_mtx("data/experimental_data/sup_fig2c.mtx")
dat = np.rot90(dat)
meta["ymax"], meta["ymin"] = meta["ymin"], meta["ymax"]
fig = plt.figure(figsize=(fig_width, fig_height))
ax = fig.add_axes([0.1, 0.2, 0.71, 0.85])
extent = (meta["xmin"], meta["xmax"], meta["ymin"], meta["ymax"])

norm = HistogramNormalize(dat, vmin=0.0, vmax=15, mixing_degree=0.8)
kwargs = dict(
    extent=extent,
    cmap="viridis",
    aspect="2.5",
    norm=norm,
    interpolation="nearest",
    origin="lower",
    vmax=norm.vmax,
)
im = ax.imshow(dat, **kwargs)
ax.set_xlabel(r"$B$ [T]")
ax.set_ylabel(r"$V_\textrm{bias}$ [$\mu$V]")
ax.set_xticks([-15, -10, -5, 0, 5])
ax.set_yticks([-4, -2, 0, 2, 4])
cax = fig.add_axes([0.83, 0.2, 0.04, 0.85])
cb = fig.colorbar(im, cax=cax, label=r"counts", extend="max")
cb.set_ticks([0, 5, 10, 15, 20, 25])

fig.savefig(
    "figures/sup_fig2c.pdf", format="pdf", dpi=150, bbox_inches="tight", pad_inches=0
)
plt.show()

# Supplementary figure 2d

In [ ]:
meta, dat = open_mtx("data/experimental_data/sup_fig2d.mtx")
dat = np.rot90(dat)
meta["ymax"], meta["ymin"] = meta["ymin"], meta["ymax"]
fig = plt.figure(figsize=(fig_width2, fig_height2))
ax = fig.add_axes([0.1, 0.2, 0.71, 0.85])
extent = (meta["xmin"], meta["xmax"], meta["ymin"], meta["ymax"])
norm = HistogramNormalize(dat, vmin=0, vmax=30, mixing_degree=0.2)
kwargs = dict(
    extent=extent,
    cmap="viridis",
    aspect="auto",
    norm=norm,
    interpolation="nearest",
    origin="lower",
    vmax=norm.vmax,
)
im = ax.imshow(dat, **kwargs)
ax.set_xlabel(r"$P$ [dBm]")
ax.set_ylabel(r"$I_\textrm{bias}$ [nA]")
ax.set_xticks([5, 10, 15, 20])
ax.set_yticks([0, 0.3, 0.6, 0.9, 1.2])
cax = fig.add_axes([0.83, 0.2, 0.04, 0.85])
cb = fig.colorbar(
    im, cax=cax, extend="max", label=r"$\mathrm{d}V/\mathrm{d}I$ [k$\Omega$]"
)
cb.set_ticks(np.arange(0, 240, 10))

fig.savefig(
    "figures/sup_fig2d.pdf", format="pdf", dpi=150, bbox_inches="tight", pad_inches=0
)
plt.show()

# Supplementary figure 2e

In [ ]:
meta, dat = open_mtx("data/experimental_data/sup_fig2e.mtx")
dat = np.rot90(dat)
meta["ymax"], meta["ymin"] = meta["ymin"], meta["ymax"]
fig = plt.figure(figsize=(fig_width2, fig_height2))
ax = fig.add_axes([0.1, 0.2, 0.71, 0.85])
extent = (meta["xmin"], meta["xmax"], meta["ymin"], meta["ymax"])
norm = HistogramNormalize(dat, vmin=0, vmax=120, mixing_degree=0.5)
kwargs = dict(
    extent=extent,
    cmap="viridis",
    aspect="auto",
    norm=norm,
    interpolation="nearest",
    origin="lower",
    vmax=norm.vmax,
)
im = ax.imshow(dat, **kwargs)
ax.set_xlabel(r"$P$ [dBm]")
ax.set_ylabel(r"$I_\textrm{bias}$ [nA]")
ax.set_xticks(np.arange(-10, 6, 2))
ax.set_yticks([0, 0.3, 0.6, 0.9])
cax = fig.add_axes([0.83, 0.2, 0.04, 0.85])
ax.set_ylim(-0.1, 1.2)
cb = fig.colorbar(
    im, cax=cax, extend="max", label=r"$\mathrm{d}V/\mathrm{d}I$ [k$\Omega$]"
)
cb.set_ticks(np.arange(0, 240, 20))

fig.savefig(
    "figures/sup_fig2e.pdf", format="pdf", dpi=150, bbox_inches="tight", pad_inches=0
)
plt.show()

# Supplementary figure 2f

In [ ]:
meta, dat = open_mtx("data/experimental_data/sup_fig2f.mtx")
dat = np.rot90(dat) / 1000
meta["ymax"], meta["ymin"] = meta["ymin"], meta["ymax"]
fig = plt.figure(figsize=(fig_width2, fig_height2))
ax = fig.add_axes([0.1, 0.2, 0.71, 0.85])
extent = (meta["xmin"], meta["xmax"], meta["ymin"], meta["ymax"])
norm = HistogramNormalize(
    dat,
    vmin=0,
    #                           vmax=200,
    mixing_degree=0.5,
)
kwargs = dict(
    extent=extent,
    cmap="viridis",
    aspect="auto",
    norm=norm,
    interpolation="nearest",
    origin="lower",
    vmax=norm.vmax,
)
im = ax.imshow(dat, **kwargs)
ax.set_xlabel(r"$P$ [dBm]")
ax.set_ylabel(r"$I_\textrm{bias}$ [nA]")
ax.set_xticks(np.arange(-10, 6, 2))
ax.set_yticks([0, 0.3, 0.6, 0.9])
cax = fig.add_axes([0.83, 0.2, 0.04, 0.85])
cb = fig.colorbar(im, cax=cax, label=r"$\mathrm{d}V/\mathrm{d}I$ [k$\Omega$]")
cb.set_ticks(np.arange(0, 40, 3))

fig.savefig(
    "figures/sup_fig2f.pdf", format="pdf", dpi=150, bbox_inches="tight", pad_inches=0
)
plt.show()

# Supplementary figure 2g

In [ ]:
meta, dat = open_mtx("data/experimental_data/sup_fig2g.mtx")
dat = np.rot90(dat)
meta["ymax"], meta["ymin"] = meta["ymin"], meta["ymax"]
fig = plt.figure(figsize=(fig_width2, fig_height2))
ax = fig.add_axes([0.1, 0.2, 0.71, 0.85])
extent = (meta["xmin"], meta["xmax"], meta["ymin"], meta["ymax"])
norm = HistogramNormalize(dat, vmin=6, vmax=11, mixing_degree=0.2)
kwargs = dict(
    extent=extent,
    cmap="viridis",
    aspect="auto",
    norm=norm,
    interpolation="nearest",
    origin="lower",
    vmax=norm.vmax,
)
im = ax.imshow(dat, **kwargs)
ax.set_xlabel(r"$P$ [dBm]")
ax.set_ylabel(r"$I_\textrm{bias}$ [nA]")
ax.set_xticks(np.arange(-10, 6, 2))
ax.set_yticks([0, 0.3, 0.6, 0.9])
cax = fig.add_axes([0.83, 0.2, 0.04, 0.85])
cb = fig.colorbar(im, cax=cax, label=r"$\mathrm{d}V/\mathrm{d}I$ [k$\Omega$]")
cb.set_ticks(np.arange(0, 30, 2))

fig.savefig(
    "figures/sup_fig2g.pdf", format="pdf", dpi=150, bbox_inches="tight", pad_inches=0
)
plt.show()

# Supplementary figure 3

In [ ]:
fig, axs = plt.subplots(
    3, 1, sharey=False, sharex=True, figsize=(fig_width, fig_height * 1.6)
)
plt.subplots_adjust(
    bottom=0.2, left=0.125, right=0.80, top=0.9, hspace=0.150, wspace=0.10
)


exp = [
    open_mtx("data/experimental_data/sup_fig3a.mtx"),
    open_mtx("data/experimental_data/sup_fig3b.mtx"),
    open_mtx("data/experimental_data/sup_fig3c.mtx"),
]


fig.text(0.00, 0.6, r"$I_\textrm{bias}$ [nA]", va="center", rotation="vertical")
for i, ax in enumerate(axs.reshape(-1)):
    meta, dat = exp[i]
    meta["ymax"], meta["ymin"] = meta["ymin"], meta["ymax"]

    extent = (
        float(meta["xmin"]),
        float(meta["xmax"]),
        float(meta["ymin"]),
        float(meta["ymax"]),
    )
    data = np.rot90(dat) / 1000

    if i == 0:
        norm = HistogramNormalize(data, vmax=10, vmin=1, mixing_degree=0.25)
        kwargs = dict(
            extent=extent,
            cmap="viridis",
            aspect="auto",
            norm=norm,
            interpolation="nearest",
            origin="lower",
        )
        im = ax.imshow(data, **kwargs)
    else:
        ax.imshow(data, **kwargs)

    ax.set_xticks(np.arange(0.5, 2, 0.5))
    ax.set_yticks([-1, 0, 1])
    ax.set_xlabel("")

    if i == 2:
        ax.set_xlabel(r"$B$ [T]")

    names = [r"(a)  $0^{\circ}$", r"(b) $45^{\circ}$", r"(c) $90^{\circ}$"]

    ax.text(
        0.77,
        0.77,
        names[i],
        horizontalalignment="left",
        verticalalignment="center",
        transform=ax.transAxes,
        fontdict={"color": "w", "size": 11},
    )

cax = fig.add_axes([0.83, 0.2, 0.04, 0.7])
cbar_ticks = np.arange(0, 25, 5)
cb = fig.colorbar(
    im, cax=cax, extend="max", label=r"$\mathrm{d}V/\mathrm{d}I$ [k$\Omega$]"
)
cb.set_ticks(cbar_ticks)
fig.savefig("figures/sup_fig3.pdf", format="pdf", dpi=150, bbox_inches="tight")
plt.show()

# Supplementary figure 5

In [ ]:
meta, dat = open_mtx("data/experimental_data/sup_fig5a.mtx")
dat = np.rot90(dat) / 1000
meta["ymax"], meta["ymin"] = meta["ymin"], meta["ymax"]
fig = plt.figure(figsize=(fig_width, fig_height))
ax = fig.add_axes([0.1, 0.2, 0.71, 0.85])
extent = (meta["xmin"], meta["xmax"], meta["ymin"], meta["ymax"])

norm = HistogramNormalize(dat, vmin=0, vmax=8, mixing_degree=0.5)
kwargs = dict(
    extent=extent,
    cmap="viridis",
    aspect="auto",
    norm=norm,
    interpolation="nearest",
    origin="lower",
    vmax=norm.vmax,
)
im = ax.imshow(dat, **kwargs)
ax.set_xlabel(r"$V_\textrm{gate}$ [V]")
ax.set_ylabel(r"$I_\textrm{bias}$ [pA]")
ax.set_xticks(np.arange(1.26, 1.33, 0.02))
ax.set_yticks(np.arange(-15, 20, 5))
cax = fig.add_axes([0.83, 0.2, 0.04, 0.85])
cb = fig.colorbar(
    im, cax=cax, extend="max", label=r"$\mathrm{d}V/\mathrm{d}I$ [k$\Omega$]"
)
cb.set_ticks(np.arange(0, 40, 2.5))

fig.savefig(
    "figures/sup_fig5a.pdf", format="pdf", dpi=150, bbox_inches="tight", pad_inches=0
)
plt.show()

In [ ]:
meta, dat = open_mtx("data/experimental_data/sup_fig5b.mtx")
dat = np.rot90(dat) / 1000
meta["ymax"], meta["ymin"] = meta["ymin"], meta["ymax"]
fig = plt.figure(figsize=(fig_width, fig_height))
ax = fig.add_axes([0.1, 0.2, 0.71, 0.85])
extent = (meta["xmin"], meta["xmax"], meta["ymin"], meta["ymax"])

norm = HistogramNormalize(dat, vmin=0, vmax=1.20, mixing_degree=0.5)
kwargs = dict(
    extent=extent,
    cmap="viridis",
    aspect="auto",
    norm=norm,
    interpolation="nearest",
    origin="lower",
    vmax=norm.vmax,
)
im = ax.imshow(dat, **kwargs)
ax.set_xlabel(r"$V_\textrm{gate}$ [V]")
ax.set_ylabel(r"$I_\textrm{bias}$ [pA]")
ax.set_xticks([0, 0.2, 0.4])
ax.set_yticks(np.arange(-5, 7.5, 2.5))
cax = fig.add_axes([0.83, 0.2, 0.04, 0.85])
cb = fig.colorbar(
    im, cax=cax, extend="max", label=r"$\mathrm{d}V/\mathrm{d}I$ [M$\Omega$]"
)
cb.set_ticks(np.arange(0, 40, 0.25))

fig.savefig(
    "figures/sup_fig5b.pdf", format="pdf", dpi=150, bbox_inches="tight", pad_inches=0
)
plt.show()

In [ ]:
meta, dat = open_mtx("data/experimental_data/sup_fig5d.mtx")
dat = np.rot90(dat)
meta["ymax"], meta["ymin"] = meta["ymin"], meta["ymax"]
fig = plt.figure(figsize=(fig_width, fig_height))
ax = fig.add_axes([0.1, 0.2, 0.71, 0.85])
extent = (meta["xmin"], meta["xmax"], meta["ymin"], meta["ymax"])

norm = HistogramNormalize(dat, vmin=1, vmax=7, mixing_degree=0.9)
kwargs = dict(
    extent=extent,
    cmap="viridis",
    aspect="auto",
    norm=norm,
    interpolation="nearest",
    origin="lower",
    vmax=norm.vmax,
)
im = ax.imshow(dat, **kwargs)
ax.set_xlabel(r"$B$ [T]")
ax.set_ylabel(r"$V_\textrm{bias}$ [$\mu$V]")
ax.set_xticks([0, 0.2, 0.4])
ax.set_yticks(np.arange(-200, 300, 100))
cax = fig.add_axes([0.83, 0.2, 0.04, 0.85])
cb = fig.colorbar(im, cax=cax, label=r"$\mathrm{d}I/\mathrm{d}V$ [$\mu$S]")
cb.set_ticks(np.arange(0, 40, 2))

fig.savefig(
    "figures/sup_fig5d.pdf", format="pdf", dpi=150, bbox_inches="tight", pad_inches=0
)
plt.show()

# Supplementary figure 6: system schematic

In [ ]:
import funcs
import types
import kwant
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import proj3d


class Arrow3D(FancyArrowPatch):
    def __init__(self, xs, ys, zs, *args, **kwargs):
        FancyArrowPatch.__init__(self, (0, 0), (0, 0), *args, **kwargs)
        self._verts3d = xs, ys, zs

    def draw(self, renderer):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))
        FancyArrowPatch.draw(self, renderer)


# Create system with infinite leads
syst_pars = dict(
    a=8,
    angle=0,
    site_disorder=False,
    holes=True,
    L=200,
    L_sc=80,
    phi=135,
    r1=50,
    r2=70,
    shape="circle",
    with_leads=True,
    with_shell=True,
    with_vlead=True,
)
syst, hopping = funcs.make_3d_wire(**syst_pars)
plt.rcParams["axes.facecolor"] = "white"
i = [i.pos for i in syst.sites][0]
params = dict(
    alpha=20,
    B_y=0,
    B_z=0,
    Delta=60,
    g=50,
    mu=50,
    orbital=True,
    V=lambda x: 0,
    B_x=0,
    **funcs.constants.__dict__
)

Deltas = [
    np.abs(syst.hamiltonian(i, i, params=params)[0, 1])
    for i, _ in enumerate(syst.sites)
]

fig = kwant.plot(
    syst,
    site_color=Deltas,
    colorbar=False,
    site_size=0.4,
    cmap="viridis",
    fig_size=(12, 9),
    show=False,
)
ax = fig.axes[0]

ax.set_xlabel("x axis", size=14)
ax.set_ylabel("y axis", size=14)
ax.set_zlabel("z axis", size=14)
ax.tick_params(axis="both", which="major", labelsize=14)
ax.tick_params(axis="both", which="minor", labelsize=14)

fig.add_axes(ax)
kwant.plotter.output_fig(fig, file="figures/system.pdf")
plt.show()

# Supplementary figure 7: 1D wire $\alpha$ vs $B_x$

In [ ]:
from glob import glob

for file in glob("data/1d_alpha_vs_B_x_T*"):
    df = pd.read_hdf(file)
    gb = df.groupby("mu")

    fig, axs = plt.subplots(
        2, 3, sharey=True, sharex=True, figsize=(2 * fig_width, 2 * fig_height)
    )
    plt.subplots_adjust(
        bottom=0.2, left=0.125, right=0.80, top=0.9, hspace=0.15, wspace=0.15
    )

    names = [
        r"(a)   $\mu=\SI{0.1}{meV}$   ",
        r"(b)   $\mu=\SI{0.3}{meV}$   ",
        r"(c)   $\mu=\SI{1}{meV}$   ",
        r"(d)   $\mu=\SI{3}{meV}$   ",
        r"(e)   $\mu=\SI{10}{meV}$   ",
        r"(f)   $\mu=\SI{30}{meV}$   ",
    ]

    for i, (ax, (mu, gr)) in enumerate(zip(axs.flatten(), gb)):
        norm = HistogramNormalize(gr.current_c.values)
        extent = (gr.B_x.min(), gr.B_x.max(), gr.alpha.min(), gr.alpha.max())
        ax.imshow(
            gr.pivot_table(index="B_x", columns="alpha", values="current_c").T.values,
            origin="lower",
            cmap="viridis",
            extent=extent,
            aspect="auto",
            norm=norm,
            interpolation="spline16",
        )

        ax.set_yticks([0, 10, 20, 30])
        ax.set_xticks([0, 0.4, 0.8])
        ax.set_xticklabels(["0", "0.7", "1.4"])

        if i in [3, 4, 5]:
            ax.set_xlabel("$B_x$ [T]")
        if i in [0, 3]:
            ax.set_ylabel(r"$\alpha$ [meV$\cdot$nm]")

        ax.text(
            0.5,
            0.90,
            names[i],
            horizontalalignment="center",
            verticalalignment="center",
            transform=ax.transAxes,
            fontdict={"color": "k", "size": 11},
        )

    fig.savefig(
        "figures/currents_1D_alpha_vs_B_x_T{:.2f}.pdf".format(df["T"].unique()[0]),
        format="pdf",
        dpi=150,
        bbox_inches="tight",
    )

# Supplementary figure 8: CPR and energy as function of $\phi$

In [ ]:
df = pd.read_hdf(
    "data/I_c(B_x)_mu10,20meV_disorder0,75meV_T0.1K_all_combinations_of_effects.hdf"
)
df = df.set_index("B_x", drop=False)
df = add_energy_gs(df)

params = ["disorder", "orbital", "g", "alpha", "mu", "salt"]
gb = df.groupby(params)

groups = [
    (
        (0, True, 50, 0, 10, 0),
        (0.02, 0.04, 0.15),
        r"$\alpha=0,$" + "\n" + r"$\mu=\SI{10}{meV}$",
    ),
    (
        (0, True, 50, 0, 10, 0),
        (0.23, 0.235, 0.24),
        r"$\alpha=0,$" + "\n" + r"$\mu=\SI{10}{meV}$",
    ),
    (
        (0, True, 50, 0, 20, 0),
        (0.025, 0.035, 0.105),
        r"$\alpha=0,$" + "\n" + r"$\mu=\SI{20}{meV}$",
    ),
    (
        (0, False, 50, 0, 10, 0),
        (0.015, 0.035, 0.095),
        r"$\bm{A}=0, \; \alpha=0,$" + "\n" + r"$\mu=\SI{10}{meV}$",
    ),
    (
        (0, False, 50, 0, 20, 0),
        (0.025, 0.07, 0.105),
        r"$\bm{A}=0, \; \alpha=0,$" + "\n" + r"$\mu=\SI{20}{meV}$",
    ),
    (
        (75, True, 50, 20, 20, 7),
        (0.06, 0.14, 0.29),
        r"$\textrm{disorder,}$" + "\n" + r"$\mu=\SI{20}{meV}$",
    ),
    (
        (75, True, 50, 20, 20, 7),
        (0.34, 0.4, 0.48),
        r"$\textrm{disorder,}$" + "\n" + r"$\mu=\SI{20}{meV}$",
    ),
]

fig, axs = plt.subplots(4, len(groups), figsize=(2.9 * fig_width, 4 * fig_height))
plt.subplots_adjust(
    bottom=0.2, left=0.125, right=0.90, top=0.9, hspace=0.35, wspace=0.5
)

for i, ax in enumerate(axs.reshape(-1)):
    key, B_xs, title = groups[i % len(groups)]
    gr = gb.get_group(key)
    ax.tick_params(axis="y", which="major", labelsize=8, pad=0, direction="out")
    ax.tick_params(axis="y", which="minor", labelsize=8, pad=0)

    if i < len(groups):
        ax.set_title(title, size=10)
        ax.plot(gr["B_x"], gr["current_c"], c="k", label=title)
        ax.set_xlim(0, 0.5)
        ax.set_xticks([0, 0.5])

    elif len(groups) <= i < 2 * len(groups):
        ax.plot(gr["B_x"], gr["phase_gs"], c="k", label=title)
        ax.set_xlim(0, 0.5)
        ax.set_xticks([0, 0.5])
        ax.set_ylim(-3.4, 3.4)
        ax.set_yticks([-np.pi, 0, np.pi])
        ax.set_yticklabels([r"$-\pi$", "$0$", r"$\pi$"])

    for c, B_x in zip(["c", "g", "m"], B_xs):
        row = gr.iloc[gr.index.get_loc(B_x, method="nearest")]
        B_x = row["B_x"]
        label = r"$B_x={:.2f}$".format(B_x)

        if i < len(groups):
            ax.scatter(row["B_x"], row["current_c"], c=c, label=title)
            ax.set_xlabel(r"$B_x$", labelpad=2)

            # Add text labels
            ax.text(
                0.98,
                0.90,
                "({})".format("abcdefghi"[i]),
                horizontalalignment="right",
                verticalalignment="center",
                transform=ax.transAxes,
                fontdict={"color": "k", "size": 11},
            )

        elif len(groups) <= i < 2 * len(groups):
            ax.scatter(row["B_x"], row["phase_gs"], c=c, label=title)
            ax.set_xlabel(r"$B_x$", labelpad=2)

        elif 2 * len(groups) <= i < 3 * len(groups):
            ax.plot(row["phases"], row["currents"], c=c, label=title)

        elif 3 * len(groups) <= i < 4 * len(groups):
            ax.plot(row["phases"], row["E"], c=c, label=title)
            phase_gs_arg = row["phase_gs_arg"]
            ax.scatter(
                row["phase_gs"], row["E"][phase_gs_arg], c=c, label=title, zorder=3
            )

            if np.abs(row.phase_gs_sort[0] - row.phase_gs_sort[1]) < 1e-10:
                arg = row.phase_gs_argsort[1]
                E_J = row.E[arg]
                phase = row.phases[arg]
                ax.scatter(phase, E_J, c=c, label=title, zorder=3)

        if i >= 2 * len(groups):
            ax.set_xticks([-3.14, 0, 3.14])
            ax.set_xticklabels([r"$-\pi$", 0, r"$\pi$"])
            ax.set_xlabel(r"$\phi$", labelpad=2)

    ax.locator_params(nbins=5, axis="y")

axs[0][0].set_ylabel(r"$I_c$ [nA]")
axs[1][0].set_ylabel(r"$\theta_{\textrm{gs}}$")
axs[2][0].set_ylabel(r"$I$ [nA]")
axs[3][0].set_ylabel(r"$E_J$ [$\mu$eV]")
fig.savefig(
    "figures/sup_cpr.pdf", format="pdf", dpi=150, bbox_inches="tight", pad_inches=0.2
)

# Supplementary figure 9: gate dependence without disorder

In [ ]:
df = pd.read_hdf("data/I_c(B_x,_V)_gate160nm_mu20meV_disorder0,75meV_T0.1K.hdf")
df["mu_gate"] = df["mu"] - df["V"]
gb = df.groupby(["disorder", "gate_size"])
print(gb.groups.keys())
gr = gb.get_group((0, 160))

fig = plt.figure(figsize=(fig_width, fig_height))
ax = fig.add_axes([0.1, 0.2, 0.71, 0.85])
extent = (gr.min()["B_x"], gr.max()["B_x"], gr.min()["mu_gate"], gr.max()["mu_gate"])

data = gr.pivot(index="B_x", columns="mu_gate", values="current_c").T

norm = HistogramNormalize(np.array(data), mixing_degree=0.5)
kwargs = dict(
    extent=extent,
    cmap="viridis",
    aspect="auto",
    norm=norm,
    origin="lower",
    interpolation="spline16",
)

im = ax.imshow(data, **kwargs)

ax.set_xlabel("Magnetic field $B$ [$T$]")
ax.set_ylabel(r"$\mu_{\textrm{gate}}$ [meV]")

cax = fig.add_axes([0.83, 0.2, 0.04, 0.85])

cb = fig.colorbar(im, cax=cax, label=r"$I_c$ [nA]")
fig.savefig(
    "figures/gate_dependence_no_disorder.pdf",
    format="pdf",
    dpi=150,
    bbox_inches="tight",
)

# Supplementary figure 10: rotation of the magnetic field

In [ ]:
df = pd.read_hdf("data/I_c(B_x)_mu20meV_rotation_of_field_in_xy_plane.hdf")
gb = df.groupby(["theta", "L", "mu", "disorder"])

theta_0 = gb.get_group((0, 160, 20, 75))
theta_45 = gb.get_group((45, 160, 20, 75))
theta_90 = gb.get_group((90, 160, 20, 75))

B_x = theta_0["B"]

fig = plt.figure(figsize=(fig_width, fig_height))
ax = fig.add_axes([0.125, 0.2, (0.95 - 0.125), (0.95 - 0.2)])

ax.plot(B_x, theta_0["current_c"], label=r"$\theta=0^{\circ}$")
ax.plot(B_x, theta_45["current_c"], label=r"$\theta=45^{\circ}$")
ax.plot(B_x, theta_90["current_c"], label=r"$\theta=90^{\circ}$")

ax.set_yticks(np.arange(0, 140, 10))
ax.set_ylim(-1, 80)
ax.set_xlabel(r"Magnetic field $|\bm{B}|$ [$T$]")
ax.set_ylabel("$I_c$ [nA]")
ax.legend(framealpha=1, ncol=2)
fig.savefig("figures/rotation_of_field.pdf", format="pdf", dpi=150, bbox_inches="tight")
plt.show()